# 特徴量エンジニアリング

In [1]:
import os
import time
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize_scalar
from scipy import signal
from sklearn.metrics import r2_score, mean_squared_error

from FuncLaserScattering import delete_distance, smoothing_cheese, skip_distance
from FuncLaserScattering import fitting, calc_diff, image2feature

In [2]:
days = "231019_MN230704"

In [5]:
%%time
# 特徴量の作成
for wave in ['green_532','red_633','nir_808', 'nir_850', 'violet_405']:
    # データの読み込み
    total_df = pd.read_csv(f"./../../Reserch/Laserscattering/experiment/Profile/{days}/Profile_{wave}.csv")

    # 距離と輝度（強度）を分割
    distance = total_df.iloc[:,0]
    cheese_df = total_df.drop('distance (mm)',axis=1)

    #####解析範囲外の領域を削除#####
    distance_30mm, cheese_df_30mm = delete_distance(distance, cheese_df)

    ######プロファイルの平滑化######
    # 0.03 mm間隔で取得し、0.5 mm〜のデータを4次元、7点で平滑化
    distance_smooth, cheese_smooth = smoothing_cheese(distance_30mm, cheese_df_30mm)
    
    #####プロファイルの間引き (1 mm間隔)#####
    # 平滑化プロファイルから1 mm間隔（0.5~29.5 mm、30点）で取得
    distance_eq, cheese_smooth_eq = skip_distance(distance_smooth, cheese_smooth)
    
    #####変化率の取得#####
    res = calc_diff(distance_eq, cheese_smooth_eq)

    #####CurveFitting#####
    eff_df = pd.DataFrame()
    for i in range(cheese_smooth_eq.shape[1]): 
        eff_temp = fitting(distance_eq, cheese_smooth_eq.iloc[:,i])
        eff_df = pd.concat([eff_df,eff_temp],axis=0)
    eff_df.index = res.columns

    # 結合の都合上転置する
    eff_df = eff_df.T

    #####画像から特徴量を抽出#####
    print(f'start image analysis for {wave}')
    # HDR1
    print("HDR1")
    image_path1 = f"./../../Reserch/Laserscattering/experiment/HDR1/{days}/{wave}"
    feature_img1 = image2feature(image_path1)
    feature_img1.index = res.columns

    # HDR2
    print("HDR2")
    image_path2 = f"./../../Reserch/Laserscattering/experiment/HDR2/{days}/{wave}"
    feature_img2 = image2feature(image_path2)
    feature_img2.index = res.columns

    # HDR3
    print("HDR3")
    image_path3 = f"./../../Reserch/Laserscattering/experiment/HDR3/{days}/{wave}"
    feature_img3 = image2feature(image_path3)
    feature_img3.index = res.columns

    # 露光合成
    print("Blend")
    image_path4 = f"./../../Reserch/Laserscattering/experiment/Blend/{days}/{wave}"
    feature_img4 = image2feature(image_path4)
    feature_img4.index = res.columns

    ####全特徴量の結合および保存#####
    feature_df = pd.concat([eff_df, res, feature_img1.T, feature_img2.T, feature_img3.T, feature_img4.T],axis=0).T
    #feature_df = pd.concat([eff_df, res],axis=0).T

    ##### データの保存 #####
    # indexに実際の距離が来るように指定
    cheese_smooth.index = distance_smooth
    cheese_smooth_eq.index = distance_eq
    # 平滑化したもの（0.03 mm間隔）
    cheese_smooth.to_csv(f'./../../Reserch/Laserscattering/experiment/results/Profile_smooth/Profile_{days}_{wave}.csv')
    # 平滑化したものから1mm間隔で間引いたもの（このdfから変化率とfitting係数を取得）
    cheese_smooth_eq.to_csv(f'./../../Reserch/Laserscattering/experiment/results/Profile_smooth/Profile_fitting_{days}_{wave}.csv')
    #  特徴量のdf
    feature_df.to_csv(f'./../../Reserch/Laserscattering/experiment/results/features/Feature_{days}_{wave}.csv')

start image analysis for violet_405
HDR1
HDR2
HDR3
Blend
CPU times: user 1h 7min 29s, sys: 49.4 s, total: 1h 8min 19s
Wall time: 1h 7min 17s
